<a href="https://colab.research.google.com/github/cholong2/Comment-Cleaner/blob/main/KoBERT_multi_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브와 연동합니다
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 필요한 모듈을 설치합니다

!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1

#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
args = {
     'train_data_path': '/content/gdrive/My Drive/Colab Notebooks/senti_train.xlsx',# Train Dataset file 
    'val_data_path': '/content/gdrive/My Drive/Colab Notebooks/senti_validation.xlsx',# Validation Dataset file 
}

In [ ]:
import pandas as pd

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

df = pd.read_excel(args["train_data_path"], sheet_name=0)

#감정, 발화만 추출
df = df.iloc[:, [ 5, 7]]
df.columns = ['label', 'document']
df['label'] = df['label'].replace(['분노', '슬픔', '불안', '상처', '당황', '기쁨'],[0.0,1.0,2.0,3.0,4.0,5.0])


#모델에 입력하기 위해 형식을 맞추기

dfls = [list(df.iloc[i,:]) for i in range(len(df))]
dfls[:10] #형식이 통일되었습니다


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule 

In [ ]:
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

##학습


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

In [ ]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dfls, test_size=0.2, random_state=123)

In [ ]:
data_train = BERTDataset(dataset_train, 1, 0, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 1, 0, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
#모델을 만들고 GPU 사용 설정
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
#옵티마이저와 손실함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#학습 과정


for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# 차후 사용을 위해 학습된 모델 저장 코드
torch.save(model.state_dict(), 'drive/My Drive//kobert_ending_finale.pt')

##평가

In [ ]:
df_val = pd.read_excel(args["val_data_path"], sheet_name=0)

#감정, 발화만 추출
df_val = df_val.iloc[:, [ 5, 7]]
df_val.columns = ['label', 'document']
df_val['label'] = df_val['label'].replace(['분노', '슬픔', '불안', '상처', '당황', '기쁨'],[0.0,1.0,2.0,3.0,4.0,5.0])


#모델에 입력하기 위해 형식을 맞추기
dfls_val = [list(df_val.iloc[i,:]) for i in range(len(df_val))]
dfls_val[:10] #형식이 통일되었습니다



data_test = BERTDataset(dfls_val, 1, 0, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
# 해당 데이터에 대해 분류를 시작
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())
  test_acc += calc_accuracy(out, label)
print('정답률: ',test_acc / (batch_id+1))